In [1]:
import h5py
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.ndimage import gaussian_filter1d

# Find the repo root (parent of current folder)
repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')) if '__file__' not in globals() else os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(repo_root)

# Add only immediate subfolders of the repo root to sys.path (much faster)
for item in os.listdir(repo_root):
    item_path = os.path.join(repo_root, item)
    if os.path.isdir(item_path):
        sys.path.append(item_path)

from read_scope_data import read_hdf5_scope_data, read_hdf5_scope_tarr, read_hdf5_all_scopes_channels
from read_hdf5 import print_hdf5_info, read_bmotion_control, print_scope_channel_descriptions

%matplotlib qt

In [46]:
ifn = r"E:\AUG2025\LP\30-LP-p24-XYplane+line-He1kG300G-6300A_2025-08-21.hdf5"
m_list, pos_array, npos, nshot = read_bmotion_control(ifn)
# print_scope_channel_descriptions(ifn, 'lpscope')
# print_hdf5_info(ifn)

Available motion groups: ['<Fenix-2>P24']
Using motion group: <Fenix-2>P24
Position array shape: (1356,)
Position array dtype: [('shot_num', '>u4'), ('x', '>f4'), ('y', '>f4')]
Structured array fields: ('shot_num', 'x', 'y')
Number of shots: 1356
Shots per position: 6
Number of unique positions: 226


In [47]:
scope_name = 'lpscope'
shot_a = 1
shot_b = 6

with h5py.File(ifn, 'r') as f:
    tarr = read_hdf5_scope_tarr(f, scope_name)
    
    plt.figure(figsize=(12, 6))
    
    shot_data = []
    for shot_num in range(shot_a, shot_b + 1):
        result = read_hdf5_all_scopes_channels(f, shot_num, False)
        c1_data = result[scope_name]['channels']['C1']
        plt.plot(tarr * 1e3, c1_data, label=f'Shot {shot_num}')
        shot_data.append(c1_data)
    
    # Calculate and plot average
    avg_data = np.mean(shot_data, axis=0)
    plt.plot(tarr * 1e3, avg_data, 'k-', linewidth=2, label='Average')
    
    plt.xlabel('Time (ms)')
    plt.ylabel('Signal')
    plt.legend()
    plt.title(f'Shots {shot_a} to {shot_b}')
    plt.tight_layout()
    plt.show()

In [48]:
with h5py.File(ifn, 'r') as f:
    # Get time array and number of time points
    tarr = read_hdf5_scope_tarr(f, scope_name)
    nt = len(tarr)

    # Prepare arrays for averaged data
    lp_data_a = np.zeros((npos, nt))
    lp_data_b = np.zeros((npos, nt))

    for i in range(npos):
        data_stack_a = []
        data_stack_b = []
        for j in range(nshot):
            shot_num = i * nshot + j + 1  # 1-based shot numbering
            result = read_hdf5_all_scopes_channels(f, shot_num, False)
            data_stack_a.append(result['lpscope']['channels']['C1'])
            data_stack_b.append(result['lpscope']['channels']['C2'])
        lp_data_a[i] = np.mean(data_stack_a, axis=0)
        lp_data_b[i] = np.mean(data_stack_b, axis=0)

In [49]:
tndx_ranges = [(55000, 60000), (65000, 70000), (75000, 80000), (85000, 90000)]
colors_list = ['red', 'blue', 'green', 'orange']

plt.figure(figsize=(10, 6))

# First subplot: time evolution at a position
plt.subplot(2, 1, 1)
plt.plot(tarr * 1e3, lp_data_a[-19])
for (tndx1, tndx2), color in zip(tndx_ranges, colors_list):
    plt.axvspan(tarr[tndx1]*1e3, tarr[tndx2]*1e3, alpha=0.2, color=color)
plt.xlabel('Time (ms)')
plt.ylabel('average Isat (V)')
plt.title('Time evolution at center of machine')

# Second subplot: profile at selected time (averaged over time ranges)
plt.subplot(2, 1, 2)
for (tndx1, tndx2), color in zip(tndx_ranges, colors_list):
    avg_range_a = np.mean(lp_data_a[-37:-1, tndx1:tndx2+1], axis=1)
    avg_range_b = np.mean(lp_data_b[-37:-1, tndx1:tndx2+1], axis=1)
    avg_combined = (avg_range_a + avg_range_b) / 2
    plt.plot(m_list[-37:-1, 0], avg_combined, color=color)

plt.xlabel('Position (cm)')
plt.ylabel('average Isat (V)')
plt.title('Density profile')

plt.tight_layout()
plt.show()

In [50]:
# Create a figure with subplots for each time range
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

nx = 9
ny = 21
xmin = m_list[0,0]
xmax = m_list[nx-1,0]
ymax = m_list[0,1]
ymin = m_list[(ny-1)*nx,1]

for idx, ((tndx1, tndx2), color) in enumerate(zip(tndx_ranges, colors_list)):
    # Average over time range
    avg_range = np.mean(lp_data_a[0:npos-37, tndx1:tndx2+1], axis=1)
    
    # Reshape to 2D grid for imshow
    data_2d = avg_range.reshape(ny, nx)
    
    # Create imshow plot
    im = axes[idx].imshow(data_2d, extent=[xmin, xmax, ymin, ymax],
                          origin='lower', aspect='auto', cmap='rainbow')
    axes[idx].set_xlabel('X Position (cm)')
    axes[idx].set_ylabel('Y Position (cm)')
    axes[idx].set_title(f'Time Range {tarr[tndx1]*1e3:.1f}-{tarr[tndx2]*1e3:.1f} ms')
    plt.colorbar(im, ax=axes[idx], label='Signal')

plt.tight_layout()
plt.show()

In [40]:
ymin

-20.0